In [1]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, Image as IPImage

from PIL import Image

from ipywidgets import interact, Dropdown, IntText

import cv2
from skimage import feature
from scipy.stats import skew, kurtosis
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.decomposition import PCA
import numpy as np

from MirrorExtractor.mirror_extractor import MirrorExtractor

from sklearn.ensemble import IsolationForest

In [19]:
def show_image(image_path, mirror_id):
    img = Image.open(image_path)
    x_coords, y_coords = extractor.get_coords(mirror_id)
    
    plt.figure(figsize=(10, 8))
    plt.plot(x_coords, y_coords, 'r-', lw=2)
    plt.scatter(x_coords[:-1], y_coords[:-1], c='cyan', s=30)
    plt.imshow(img)
    plt.axis('off')
    text_title = image_path.split('/')[-1] + "mirror id:" + str(mirror_id)
    plt.title(text_title)
    plt.show()

def extract_one_mirror_from_images(img_path_list, mirror_id, extractor):
    mirror_list = []
    for path in img_path_list:
        img = np.array(Image.open(path).convert('RGB'))
        x_coords, y_coords = extractor.get_coords(mirror_id)
        cropped = extractor.extract_polygon_region_cv2(img, x_coords, y_coords)
        mirror_list.append(cropped)
    return mirror_list

def get_one_mirror_features(mirror_img):
    brightness = np.mean(mirror)
    brightness_std = np.std(mirror)
    return brightness
    
def compute_mirror_features(mirror_list):
    """
    Oblicza cechy dla każdego lustra
    """
    features_dict = {}
    brightness_list = []
    brightness_std_list = []
    
    
    for mirror in mirror_list:
        brightness_list.append(np.mean(mirror))
        brightness_std_list.append(np.std(mirror))
    
    features_dict["brightness"] = brightness_list
    features_dict["brightness_std"] = brightness_std_list
    return features_dict

## 1. Build base model with good images

In [20]:
extractor = MirrorExtractor("/home/pgliwny/Praca/Computer_vision_for_MAGIC/data/crossings_points.pkl")

In [21]:
img_list = glob.glob("/home/pgliwny/Praca/Computer_vision_for_MAGIC/data/webcam_useful_image/webcam_useful_images/*.jpg")
print(img_list[:5])
mirror_list_150 = extract_one_mirror_from_images(img_list[:], 150, extractor)
mirror_list_50 = extract_one_mirror_from_images(img_list[:], 50, extractor)

['/home/pgliwny/Praca/Computer_vision_for_MAGIC/data/webcam_useful_image/webcam_useful_images/image_2024-05-09_1500.jpg', '/home/pgliwny/Praca/Computer_vision_for_MAGIC/data/webcam_useful_image/webcam_useful_images/image_2023-01-04_1600.jpg', '/home/pgliwny/Praca/Computer_vision_for_MAGIC/data/webcam_useful_image/webcam_useful_images/image_2024-05-07_1600.jpg', '/home/pgliwny/Praca/Computer_vision_for_MAGIC/data/webcam_useful_image/webcam_useful_images/image_2024-05-04_1000.jpg', '/home/pgliwny/Praca/Computer_vision_for_MAGIC/data/webcam_useful_image/webcam_useful_images/image_2024-05-21_1700.jpg']


In [23]:
features_150 = compute_mirror_features(mirror_list_150)

In [26]:
features_150.keys()

dict_keys(['brightness', 'brightness_std'])

In [22]:
mirror_id_list = [15, 50, 100]

for mirror_id in mirror_id_list:
    print(mirror_id)

15
50
100


In [17]:
mirror_status = np.ones(289)